In [25]:
# importing the module
import cv2
import math
from win32api import GetSystemMetrics
import time
import numpy as np
import os

In [26]:
# Image path
imagePath = 'C:\\Users\\Salil kulkarni\\Desktop\\TARQ\\ParkingApp\\Images'
imageName = "map3"
resizedImageName = imageName + "_resized"

In [27]:
selectedPoints = []
def click_event_selectedpoints(event, x, y, flags, params):

# checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        selectedPoints.append([x,y])

def get_resized_for_display_img(img):
    screen_w, screen_h = GetSystemMetrics(0), GetSystemMetrics(1)
    #print("screen size",screen_w, screen_h)
    h,w,channel_nbr = img.shape
    # img get w of screen and adapt h
    h = h * (screen_w / w)
    w = screen_w
    if h > screen_h: #if img h still too big
        # img get h of screen and adapt w
        w = w * (screen_h / h)
        h = screen_h
    w, h = w*0.9, h*0.9 # because you don't want it to be that big, right ?
    w, h = int(w), int(h) # you need int for the cv2.resize
    return cv2.resize(img, (w, h))

def get_MP_FromBuildings_ButAlsoText(img):

    # Upper and lower color limit
    low_yellow = (239,248,253)
    high_yellow = (243,252,255)

    low_gray = (241,241,241)
    high_gray = (244,243,241)

    # create masks
    yellow_mask = cv2.inRange(img, low_yellow, high_yellow )
    gray_mask = cv2.inRange(img, low_gray, high_gray)

    # combine masks
    combined_mask = cv2.bitwise_or(yellow_mask, gray_mask)
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_DILATE,kernel)

    # findcontours
    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    output = img.copy()

    for c in cnts:
        area = cv2.contourArea(c)
        if area > 500:
            for eps in np.linspace(0.001, 0.005, 10):
                # approximate the contour
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, eps * peri, True)
            
            # draw the approximated contour on the image  
            cv2.drawContours(output, [approx], -1, (0, 255, 0), thickness=1)
            # cv2.drawContours(img, [c], -1, (255,0,255), thickness=1)


    # write points in mounting list
    mountingPoints = []
    for y in range(len(img)):
        for x in range(len(img[y])):
            b = img[y, x, 0]
            g = img[y, x, 1]
            r = img[y, x, 2]
            if(b == 255 and g == 0 and r == 255):
                mountingPoints.append((x,y))

    # return mountingPoints
    return output

In [28]:
# reading the resized the image
img = cv2.imread(os.path.join(imagePath, resizedImageName+".PNG"), 1)

image = img.copy()
# convert image to HSV
#hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

#will show black and white image of borders of the buildings. Click on this image at different points to encircle a 
#a structure for corner detection. Coordinates will not be displayed. Coordinates will be stored and resultant mask will
#be displayed.
cntr_img = get_MP_FromBuildings_ButAlsoText(img)

cv2.imshow("cntr trace", cntr_img)
cv2.waitKey(0)
cv2.destroyWindow('cntr trace')

In [29]:
# Upper and lower color limit
low_yellow = (239,248,253)
high_yellow = (243,252,255)

low_gray = (241,241,241)
high_gray = (244,243,241)

# create masks
yellow_mask = cv2.inRange(img, low_yellow, high_yellow )
gray_mask = cv2.inRange(img, low_gray, high_gray)

# combine masks
combined_mask = cv2.bitwise_or(yellow_mask, gray_mask)
kernel = np.ones((3,3), dtype=np.uint8)
combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_DILATE,kernel)

#     # combine masks

original_frame = cv2.imread(os.path.join(imagePath, resizedImageName+".PNG"), 1)
# original_frame = get_resized_for_display_img(original_frame)
points = np.array(selectedPoints)
blank = np.zeros(original_frame.shape[:2], dtype='uint8')
poly_pts = np.array(points, dtype='uint32')
poly_pts = poly_pts.reshape((-1, 1, 2))
polymask = cv2.fillPoly(blank, pts=poly_pts,color=255)
masked = cv2.bitwise_and(original_frame,original_frame,mask=polymask)
masked = cv2.bitwise_and(masked,masked,mask=combined_mask)


#will display masked quadrilateral whose points were selected prior
cv2.imshow("out", masked)
cv2.waitKey(0)
cv2.destroyWindow('out')

In [30]:
operatedImage = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)

# modify the data type
# setting to 32-bit floating point
# operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) #uncomment if you feel image is noisy (not needed)
operatedImage = np.float32(operatedImage)

# apply the cv2.cornerHarris method
# to detect the corners with appropriate
# values as input parameters
dest = cv2.cornerHarris(operatedImage, 2, 5, 0.07)

# Results are marked through the dilated corners
dest = cv2.dilate(dest, None)

# Reverting back to the original image,
# with optimal threshold value
original_frame[dest > 0.01 * dest.max()]=[0, 255, 0]

# the window showing output image with corners
cv2.imshow('Image with Borders', original_frame)

#     # De-allocate any associated memory usage
cv2.waitKey(0)
cv2.destroyAllWindows()


In [49]:
def detectCorner(image):

    # making a copy of the image to have the original image untouched in main loop
    imageSub = image.copy()
    original_frame = image.copy()

    # convet to gray and perform Harris corner detection
    gray = cv2.cvtColor(imageSub,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    og = cv2.cvtColor(original_frame,cv2.COLOR_BGR2GRAY)
    og = np.float32(og)
    dst = cv2.cornerHarris(gray,2,3,0.04)
    dest = cv2.cornerHarris(og, 2, 5, 0.07)

    # result is dilated for marking the corners, not important
    dst = cv2.dilate(dst,None)
    dest = cv2.dilate(dest, None)

    # threshold for an optimal value, it may vary depending on the image.
    imageSub[dst>0.01*dst.max()]=[0,0,255]
    # og[dest > 0.01 * dest.max()]=[0, 0, 255]

    imageSub = og-imageSub

    return imageSub

In [50]:
# reading the resized the image
img = cv2.imread(os.path.join(imagePath, resizedImageName+".PNG"), 1)

cntr_img = detectCorner(img)

cv2.imshow("cntr trace", cntr_img)
cv2.waitKey(0)
cv2.destroyWindow('cntr trace')

ValueError: operands could not be broadcast together with shapes (617,1382) (617,1382,3) 